# **Instalamos las librerias necesarias**

In [ ]:
! pip install ase nglview
! curl -LJO https://raw.githubusercontent.com/emartineznunez/Master_Quimica/master/Google_colab/FA.xyz

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

# **Visualización de moléculas**

In [ ]:
#The database is here: https://wiki.fysik.dtu.dk/ase/ase/build/build.html
from ase.build import molecule
from ase.visualize import view

water = molecule('H2O')
FA = molecule('HCOOH')
C60 = molecule('C60')
cyclobutene = molecule('cyclobutene')
view(water, viewer='x3d')

In [ ]:
view(FA, viewer='x3d')

In [ ]:
view(C60, viewer='x3d')

In [ ]:
view(cyclobutene, viewer='x3d')

In [ ]:
from ase.build import graphene_nanoribbon
from ase.visualize import view

gnr1 = graphene_nanoribbon(3, 4, type='armchair', saturated=True,
                               vacuum=3.5)
gnr2 = graphene_nanoribbon(2, 6, type='zigzag', saturated=True,
                           C_H=1.1, C_C=1.4, vacuum=3.0,
                           magnetic=True, initial_mag=1.12)
view(gnr1, viewer='ngl')

In [ ]:
view(gnr2, viewer='ngl')

In [ ]:
from ase.build import nanotube
from ase.visualize import view
from ase.io import write

cnt1 = nanotube(6, 0, length=4)
cnt2 = nanotube(3, 3, length=6, bond=1.4, symbol='Si')

view(cnt1, viewer='x3d')

In [ ]:
view(cnt2, viewer='x3d')

In [ ]:
from ase.build import fcc111
from ase.visualize import view
from ase.io import write

#https://wiki.fysik.dtu.dk/ase/ase/build/surface.html?highlight=fcc111#ase.build.fcc111

slab = fcc111('Al', size=(3,3,2))
view(slab,viewer='x3d')

In [ ]:
# ontop,fcc,bridge,hcp

# https://wiki.fysik.dtu.dk/ase/ase/build/surface.html?highlight=fcc111#ase.build.fcc111

from ase.build import fcc111,add_adsorbate
from ase.visualize import view
from ase.io import write

slab = fcc111('Al', size=(3,3,2))
add_adsorbate(slab,'H',1.5,'hcp')

view(slab,viewer='x3d')

# **Optimización**

In [ ]:
#with the Powershell prompt
#cd Desktop
# ase gui H2O.traj
# show forces, bonds
from ase import Atoms
from ase.units import kJ,mol
from ase.build import molecule
from ase.optimize import BFGS, FIRE
from ase.calculators.emt import EMT
from ase.io import read
from ase.visualize import view
import numpy as np
import warnings
warnings.filterwarnings("ignore")

n2 = molecule('N2')

mu = n2.get_masses()[0] / 2 / mol * 0.001

natoms = len(n2)

n2.set_calculator(EMT())
dyn = BFGS(n2, trajectory='N2.traj')
dyn.run(fmax=0.001)

delta=0.001

n2.positions[0][2] = n2.get_positions()[0][2] + delta
g1 = abs(n2.get_forces()[0][2])

frec = 1 / (2 * np.pi) * np.sqrt(g1/delta / kJ * 1000 / mu) / 2.9979

print('Vibrational frequency (cm-1):',frec)

configs = read('N2.traj@0:')
view(configs, viewer='ngl')

In [ ]:
from ase import *
from ase.units import kB
from ase.optimize import LBFGS
from ase.calculators.emt import EMT
from ase.optimize.basin import BasinHopping
import numpy as np

d = 0.9575
t = np.pi / 180 * 104.51
water = Atoms('H2O',
              positions=[(d, 0, 0),
                         (d * np.cos(t), d * np.sin(t), 0),
                         (0, 0, 0)],
              calculator=EMT())

bh = BasinHopping(atoms=water,         # the system to optimize
                  temperature=100 * kB, # 'temperature' to overcome barriers
                  dr=0.5,               # maximal stepwidth
                  optimizer=LBFGS,      # optimizer to find local minima
                  fmax=0.1,             # maximal force for the optimizer
                  )

In [ ]:
#with the Powershell prompt
#cd Desktop
# ase gui H2O.traj
# show forces, bonds
from ase import Atoms
from ase.optimize import BFGS, FIRE
from ase.calculators.emt import EMT
from ase.io import read
from ase.visualize import view
import numpy as np
import warnings
warnings.filterwarnings("ignore")
d = 0.9575
t = np.pi / 180 * 104.51
water = Atoms('H2O',
              positions=[(d, 0, 0),
                         (d * np.cos(t), d * np.sin(t), 0),
                         (0, 0, 0)],
              calculator=EMT())
dyn = BFGS(water, trajectory='H2O.traj')
#dyn = FIRE(water, trajectory='H2O.traj')

dyn.run(fmax=0.05)
configs = read('H2O.traj@0:')
view(configs, viewer='ngl')

In [ ]:
# Test using mopac

from ase import Atoms
from ase.build import molecule
from ase.optimize import BFGS, FIRE
from ase.calculators.emt import EMT
from ase.io import read
from ase.visualize import view
import numpy as np
import warnings
warnings.filterwarnings("ignore")

#Method 1: read xyz file
FA = read('FA.xyz')
FA.calc = EMT()

#Method 2: construct molecule with Atoms
d = 0.9575
t = np.pi / 180 * 104.51
water = Atoms('H2O',
              positions=[(d, 0, 0),
                         (d * np.cos(t), d * np.sin(t), 0),
                         (0, 0, 0)],
              calculator=EMT())


#Method 3: get the molecule from database
AcH = molecule('CH3COOH',calculator=EMT())

#Change the name of the sysmte below
dyn = BFGS(FA, trajectory='H2O.traj')

dyn.run(fmax=0.05)
configs = read('H2O.traj@0:')
view(configs, viewer='ngl')

# **Termoquímica**
https://wiki.fysik.dtu.dk/ase/ase/thermochemistry/thermochemistry.html

In [ ]:
from ase.build import molecule
from ase.calculators.emt import EMT
from ase.optimize import QuasiNewton
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo

atoms = molecule('NH3')
atoms.set_calculator(EMT())
dyn = QuasiNewton(atoms)
dyn.run(fmax=0.01)
potentialenergy = atoms.get_potential_energy()

vib = Vibrations(atoms)
vib.run()
vib_energies = vib.get_energies()
vib.clean()

thermo = IdealGasThermo(vib_energies=vib_energies,
                        potentialenergy=potentialenergy,
                        atoms=atoms,
                        geometry='nonlinear',
                        symmetrynumber=3, spin=0)
G = thermo.get_gibbs_energy(temperature=298.15, pressure=101325.)

# **Energías de disociación**

In [ ]:
from ase import Atoms
from ase.calculators.emt import EMT

atom = Atoms('N', calculator=EMT())
e_atom = atom.get_potential_energy()

d = 1.1
molecule = Atoms('2N', [(0., 0., 0.), (0., 0., d)])
molecule.set_calculator(EMT())
e_molecule = molecule.get_potential_energy()

e_atomization = e_molecule - 2 * e_atom

print('Nitrogen atom energy: %5.2f eV' % e_atom)
print('Nitrogen molecule energy: %5.2f eV' % e_molecule)
print('Dissociation energy: %5.2f eV' % -e_atomization)

In [ ]:
%matplotlib inline
from ase import Atoms
from ase.calculators.emt import EMT
from matplotlib import pyplot
import numpy as np

atom = Atoms('N', calculator=EMT())
e_atom = atom.get_potential_energy()

e = []
r = []
for d in np.linspace(0.45,10,100):
    molecule = Atoms('2N', [(0., 0., 0.), (0., 0., d)])
    molecule.set_calculator(EMT())
    ei = molecule.get_potential_energy() - 2 *e_atom
    e.append(ei)
    r.append(d)

pyplot.plot(r ,e ,'-')
pyplot.xlabel("N-N distance (Angstroms)")
pyplot.ylabel("Energy (eV)")
pyplot.title("Potential energy curve (N2)")

# **Dinámica molecular**

In [ ]:
from ase.lattice.cubic import FaceCenteredCubic
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase import units
from ase.calculators.emt import EMT
from ase.constraints import FixCom
from ase.build import molecule
from ase.io import read
from ase.visualize import view
import warnings
warnings.filterwarnings("ignore")
from google.colab import output
output.enable_custom_widget_manager()

#size*size*size is the number of times the unit cell is repeated
size=2
# Set up the surface
atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          symbol='Cu',
                          size=(size, size, size),
                          pbc=True)

# Describe the interatomic interactions with the Effective Medium Theory
atoms.set_calculator(EMT())


# Set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(atoms, 300 * units.kB)

c=FixCom()
atoms.set_constraint(c)
# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(atoms, 5 * units.fs, trajectory='md.traj')  # 5 fs time step.

# Now run the dynamics
dyn.run(100)
configs = read('md.traj@0:')
view(configs, viewer='ngl')

In [ ]:
from ase.lattice.cubic import FaceCenteredCubic
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase import units
from ase.calculators.emt import EMT
from ase.constraints import FixCom
from ase.build import molecule
from ase.visualize import view
from ase.io import read
import warnings
warnings.filterwarnings("ignore")
from google.colab import output
output.enable_custom_widget_manager()

#size*size*size is the number of times the unit cell is repeated
n2=molecule('N2')
n2.set_calculator(EMT())


# Set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(n2, 300 * units.kB)

c=FixCom()
n2.set_constraint(c)
# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(n2, 5 * units.fs, trajectory='md.traj')  # 5 fs time step.

# Now run the dynamics
dyn.run(500)
configs = read('md.traj@0:')
view(configs, viewer='ngl')

# **Ecuación de estado (sólidos)**
https://en.wikipedia.org/wiki/Murnaghan_equation_of_state

https://wiki.fysik.dtu.dk/ase/tutorials/eos/eos.html

In [ ]:
%matplotlib inline
import numpy as np

from ase import Atoms
from ase.io.trajectory import Trajectory
from ase.calculators.emt import EMT
from ase.io import read
from ase.units import kJ
from ase.eos import EquationOfState
from ase.visualize import view
import warnings
warnings.filterwarnings("ignore")

a = 4.0  # approximate lattice constant
b = a / 2
ag = Atoms('Ag',
           cell=[(0, b, b), (b, 0, b), (b, b, 0)],
           pbc=1,
           calculator=EMT())  # use EMT potential
cell = ag.get_cell()
traj = Trajectory('Ag.traj', 'w')
for x in np.linspace(0.95, 1.05, 5):
    ag.set_cell(cell * x, scale_atoms=True)
    ag.get_potential_energy()
    traj.write(ag)

configs = read('Ag.traj@0:5')  # read 5 configurations
# Extract volumes and energies:
volumes = [ag.get_volume() for ag in configs]
energies = [ag.get_potential_energy() for ag in configs]
eos = EquationOfState(volumes, energies)
v0, e0, B = eos.fit()
#print(B / kJ * 1.0e24, 'GPa')
eos.plot('Ag-eos.png')